<a href="https://colab.research.google.com/github/bochendong/giao_bochen/blob/main/unet_cifa10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import torchvision
from torchvision import datasets, transforms, models
import numpy as np
import random
from torchvision.utils import save_image
import os
from torch.optim.lr_scheduler import StepLR
import glob
import cv2
import math

In [2]:
# Hyperparameters
DNN_PATH = 'model_weight/dnn.pth'

MNIST = False
CIFAR10 = True

BATCH_SIZE = 256
learning_rate = 1e-2
num_epochs = 30
Train_BASE_DNN = True

if (os.path.exists(DNN_PATH)) == True:
    Train_BASE_DNN = False

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
if (os.path.exists("./output")) == False:
    os.mkdir("output")

if (os.path.exists("./model_weight")) == False:
    os.mkdir("model_weight")

for epoch in range (num_epochs):
    if (os.path.exists("./output/%03d" % epoch)) == False:
        os.mkdir("./output/%03d" % epoch)
    else:
        files = glob.glob("./output/%03d/*.png" % epoch)

        for f in files:
          os.remove(f)

In [4]:
if (MNIST):
    train_dataset = datasets.MNIST('data', train=True, download=True, 
                       transform=transforms.Compose([
                           transforms.Resize(32),
                           transforms.ToTensor()
                       ]))
    test_dataset =  datasets.MNIST('data', train=False, transform=transforms.Compose([
                           transforms.Resize(32),
                           transforms.ToTensor()
                       ]))
elif (CIFAR10):
    train_dataset = datasets.CIFAR10('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize(32),
                           transforms.ToTensor()
                       ]))

    test_dataset = datasets.CIFAR10('data', train=False, download=True,
                       transform=transforms.Compose([
                           transforms.Resize(32),
                           transforms.ToTensor()
                       ]))
    

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Files already downloaded and verified
Files already downloaded and verified


In [5]:
label_distribution = np.bincount(train_dataset.targets)
print("Label distribution:", label_distribution)

Label distribution: [5000 5000 5000 5000 5000 5000 5000 5000 5000 5000]


In [6]:
class VGG11(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG11, self).__init__()
        if MNIST:
            in_channal = 1
        else:
            in_channal = 3
            
        self.features = nn.Sequential(
            nn.Conv2d(in_channal, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.classifier = nn.Sequential(
            nn.Linear(512, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(4096, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [7]:
if (Train_BASE_DNN):
    dnn_model = VGG11().cuda()
    dnn_criterion = nn.CrossEntropyLoss()
    dnn_optimizer = torch.optim.Adam(dnn_model.parameters(), lr=1e-5)

    print("Training DNN classifier...")
    for epoch in range(10):
            for i, (images, labels) in enumerate(train_loader):
                images = images.cuda()
                labels = labels.cuda()

                # Forward pass
                outputs = dnn_model(images)
                loss = dnn_criterion(outputs, labels)

                # Backward pass and optimization
                dnn_optimizer.zero_grad()
                loss.backward()
                dnn_optimizer.step()

    print("DNN classifier training complete.")
    torch.save(dnn_model.state_dict(), DNN_PATH)

In [8]:
class UNet(nn.Module):
    def __init__(self):
        super(UNet, self).__init__()
        self.activate = nn.ReLU(inplace=True)
        self.pool = nn.MaxPool2d((2, 2))
        self.dropout = nn.Dropout(p=0.5)
        self.sigmod = nn.Sigmoid ()
        self.label_embedding = nn.Embedding(10, 512)

        if MNIST:
            in_channal, out_channel = 1, 1
        else:
            in_channal, out_channel = 3, 3

        self.encoder_1 = nn.Sequential(
            nn.Conv2d(in_channal, 64, 3, padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, 3, padding= 1),
            nn.ReLU(inplace=True),
        )

        self.encoder_2 = nn.Sequential(
            nn.Conv2d(64, 128, 3, padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, 3, padding= 1),
            nn.ReLU(inplace=True),
        )

        self.encoder_3 = nn.Sequential(
            nn.Conv2d(128, 256, 3, padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, 3, padding= 1),
            nn.ReLU(inplace=True),
        )

        self.encoder_4 = nn.Sequential(
            nn.Conv2d(256, 512, 3, padding= 1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, 3, padding= 1),
            nn.ReLU(inplace=True),
        )
        
        self.middle_1_0 = nn.Conv2d(1024, 1024, 3, padding= 1)
        self.middle_1_1 = nn.Conv2d(1024, 1024, 3, padding= 1)
        
       
        self.deconv4_0 = nn.ConvTranspose2d(1536, 512, 3, stride=(2,2), padding = 1, output_padding = 1)
        self.uconv4_1 = nn.Conv2d(1024, 512, 3, padding= 1) 
        self.uconv4_2 = nn.Conv2d(512, 512, 3, padding= 1)

        self.deconv3_0 = nn.ConvTranspose2d(512, 512, 3, stride=(2,2), padding = 1, output_padding = 1)
        self.uconv3_1 = nn.Conv2d(768, 256, 3, padding= 1) 
        self.uconv3_2 = nn.Conv2d(256, 256, 3, padding= 1)

        self.deconv2_0 = nn.ConvTranspose2d(256, 512, 3, stride=(2,2), padding = 1, output_padding = 1)
        self.uconv2_1 = nn.Conv2d(640, 128, 3, padding= 1) 
        self.uconv2_2 = nn.Conv2d(128, 128, 3, padding= 1)

        self.deconv1_0 = nn.ConvTranspose2d(128, 512, 3, stride=(2,2), padding = 1, output_padding = 1)
        self.uconv1_1 = nn.Conv2d(576, 192, 3, padding= 1) 
        self.uconv1_2 = nn.Conv2d(192, 192, 3, padding= 1)

  
        self.out_layer = nn.Conv2d(192, out_channel, 1)

 
    def forward(self, x, input_labels, target_labels):
        conv1 = self.encoder_1(x)
        pool1 = self.pool(conv1)
        pool1 = self.dropout(pool1)

        conv2 = self.encoder_2(pool1)
        pool2 = self.pool(conv2)
        pool2 = self.dropout(pool2)

        conv3 = self.encoder_3(pool2)
        pool3 = self.pool(conv3)
        pool3 = self.dropout(pool3)

        conv4 = self.encoder_4(pool3)
        pool4 = self.pool(conv4)
        encoder_out = self.dropout(pool4)

        input_label_embedding = self.label_embedding(input_labels).view(input_labels.size(0), 512, 1, 1)
        x1 = torch.cat([encoder_out, input_label_embedding.expand_as(encoder_out)], dim=1)

        convm = self.middle_1_0(x1)
        convm = self.activate(convm)
        convm = self.middle_1_1(convm)
        x2 = self.activate(convm)

        target_label_embedding = self.label_embedding(target_labels).view(target_labels.size(0), 512, 1, 1)
        x2 = torch.cat([x2, target_label_embedding.expand(x2.size(0), 512, x2.size(2), x2.size(3))], dim=1)

        deconv4 = self.deconv4_0(x2)
        uconv4 = torch.cat([deconv4, conv4], 1)   # (None, 4, 4, 1024)
        uconv4 = self.dropout(uconv4)
        uconv4 = self.uconv4_1(uconv4)            # (None, 4, 4, 512)
        uconv4 = self.activate(uconv4)
        uconv4 = self.uconv4_2(uconv4)            # (None, 4, 4, 512)
        uconv4 = self.activate(uconv4)

        deconv3 = self.deconv3_0(uconv4)          # (None, 8, 8, 512)
        uconv3 = torch.cat([deconv3, conv3], 1)   # (None, 8, 8, 768)
        uconv3 = self.dropout(uconv3)
        uconv3 = self.uconv3_1(uconv3)            # (None, 8, 8, 256)
        uconv3 = self.activate(uconv3)
        uconv3 = self.uconv3_2(uconv3)            # (None, 8, 8, 256)
        uconv3 = self.activate(uconv3)
        
        deconv2 = self.deconv2_0(uconv3)          # (None, 16, 16, 512)
        uconv2 = torch.cat([deconv2, conv2], 1)   # (None, 16, 16, 640)
        uconv2 = self.dropout(uconv2)
        uconv2 = self.uconv2_1(uconv2)            # (None, 16, 16, 128)
        uconv2 = self.activate(uconv2)
        uconv2 = self.uconv2_2(uconv2)            # (None, 16, 16, 128)
        uconv2 = self.activate(uconv2)

        deconv1 = self.deconv1_0(uconv2)          # (None, 32, 32, 512)
        uconv1 = torch.cat([deconv1, conv1], 1)   # (None, 32, 32, 576)
        uconv1 = self.dropout(uconv1)
        uconv1 = self.uconv1_1(uconv1)            # (None, 32, 32, 192)
        uconv1 = self.activate(uconv1)
        uconv1 = self.uconv1_2(uconv1)            # (None, 32, 32, 192)
        uconv1 = self.activate(uconv1)

        out = self.out_layer(uconv1)
        out = self.sigmod(out)

        return out

In [9]:
'''
dataset_iter = iter(train_loader)
test_img, test_label = next(dataset_iter)

target_labels = (test_label + torch.randint(1, 9, size=(BATCH_SIZE,))) % 10

model = UNet()
model(test_img, test_label, target_labels).size()
'''


'\ndataset_iter = iter(train_loader)\ntest_img, test_label = next(dataset_iter)\n\ntarget_labels = (test_label + torch.randint(1, 9, size=(BATCH_SIZE,))) % 10\n\nmodel = UNet()\nmodel(test_img, test_label, target_labels).size()\n'

In [10]:
alpha = 0.1
beta = 0.05

step_size = 6   # Decrease the learning rate every 10 epochs
gamma = 0.2      # Multiply the learning rate by 0.1 at each step

# Initialize model, loss, and optimizer
model = UNet().cuda()
dnn_model = VGG11().cuda()
dnn_model.load_state_dict(torch.load(DNN_PATH))
dnn_criterion = nn.CrossEntropyLoss()

# Freeze the DNN classifier weights
for param in dnn_model.parameters():
    param.requires_grad = False
    
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=step_size, gamma=gamma)

def perceptual_loss(vgg_model, input_images, output_images):
    feature_layers = [vgg_model.features[i] for i in range(len(vgg_model.features))]
    feature_extractor = nn.Sequential(*feature_layers[:-1]).cuda()
    
    input_features = feature_extractor(input_images)
    output_features = feature_extractor(output_images)
    
    return nn.functional.mse_loss(input_features, output_features)

# Erode the input images to remove the digit information
def erode_images_1_channel(images):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    eroded_images = []
    for image in images:
        gray_image = image.squeeze(0).detach().cpu().numpy()
        eroded_image = cv2.erode(gray_image, kernel, iterations=1)
        eroded_images.append(eroded_image)
    
    eroded_images_np = np.array(eroded_images)
    return torch.tensor(eroded_images_np).unsqueeze(1).cuda()


def erode_images_3_channel(images):
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3))
    eroded_images = []
    for image in images:
        rgb_image = image.permute(1, 2, 0).detach().cpu().numpy()
        eroded_image = np.zeros_like(rgb_image)
        for i in range(3):
            eroded_image[:, :, i] = cv2.erode(rgb_image[:, :, i], kernel, iterations=1)
        eroded_images.append(eroded_image)
    
    eroded_images_np = np.array(eroded_images).transpose(0, 3, 1, 2)
    return torch.tensor(eroded_images_np).cuda()


def generate_synthetic_digits_1_channel(digit, count):
    digit_indices = np.where(train_dataset.targets.cpu() == digit.cpu())[0]
    selected_indices = np.random.choice(digit_indices, count, replace=True)
    synthetic_digits = torch.stack([train_dataset[i][0] for i in selected_indices])
    return synthetic_digits

def generate_synthetic_digits_cifa_10(label, count):
    label_indices = np.where(np.array(train_dataset.targets) == label.cpu().numpy())[0]
    if len(label_indices) == 0:
        raise ValueError(f"No samples found for label {label}")
    
    selected_indices = np.random.choice(label_indices, count, replace=True)
    synthetic_images = torch.stack([train_dataset[i][0] for i in selected_indices])
    return synthetic_images


# Training loop
for epoch in range(num_epochs):
    print('e:' , epoch)
    for i, (images, labels) in enumerate(train_loader):
        images = images.cuda()
        labels = labels.cuda()

        target_labels = (labels + torch.randint(1, 9, size=(BATCH_SIZE,)).cuda()) % 10

        outputs = model(images, labels, target_labels)
        
        # Generate target images (same digit as target labels)
        if MNIST:
            eroded_images = erode_images_1_channel(images)
        else:
            eroded_images = erode_images_3_channel(images)

        if MNIST:
            synthetic_target_images = torch.cat([generate_synthetic_digits_1_channel(d, 1) 
                                                    for d in target_labels]).cuda()
        else:
            synthetic_target_images = torch.cat([generate_synthetic_digits_cifa_10(l, 1) 
                                                    for l in target_labels]).cuda()

        target_images = (eroded_images + synthetic_target_images) / 2

        # Compute loss
        reconstruction_loss = criterion(outputs, target_images)
        classification_loss = dnn_criterion(dnn_model(outputs), target_labels)
        p_loss = perceptual_loss(dnn_model, images, outputs)

        loss = reconstruction_loss + alpha * classification_loss + beta * p_loss

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 30 == 0:
            print('loss:' , loss.data.cpu().numpy(), 'recon loss:', reconstruction_loss.data.cpu().numpy(), 
                  'dnn loss:', alpha * classification_loss.data.cpu().numpy(), 'p loss:', beta * p_loss.data.cpu().numpy())
            save_image(outputs.data, './output/%03d/%04d_recon.png' % ( epoch, i))
            save_image(images.data, './output/%03d/%04d_img.png' % ( epoch, i))
            save_image(target_images.data, './output/%03d/%04d_target.png' % ( epoch, i))
    scheduler.step()


e: 0
loss: 0.7903534 recon loss: 0.31846762 dnn loss: 0.46265721321105957 p loss: 0.009228571504354478
loss: 0.7925441 recon loss: 0.3194053 dnn loss: 0.4636389255523682 p loss: 0.009499900788068772
loss: 0.78501225 recon loss: 0.3197832 dnn loss: 0.4560594081878662 p loss: 0.009169575572013856
loss: 0.78742003 recon loss: 0.3168581 dnn loss: 0.4612785339355469 p loss: 0.009283445030450822
loss: 0.7675872 recon loss: 0.316833 dnn loss: 0.4415956020355225 p loss: 0.009158601611852646
loss: 0.7977445 recon loss: 0.31738627 dnn loss: 0.4711233139038086 p loss: 0.009234914928674698
e: 1
loss: 0.8094578 recon loss: 0.31446064 dnn loss: 0.48580255508422854 p loss: 0.009194599092006683
loss: 0.7862124 recon loss: 0.32032585 dnn loss: 0.4566027641296387 p loss: 0.009283699840307236
loss: 0.8058366 recon loss: 0.3215037 dnn loss: 0.47510814666748047 p loss: 0.00922478660941124
loss: 0.8014609 recon loss: 0.31559658 dnn loss: 0.4764597415924072 p loss: 0.009404592216014862
loss: 0.74308985 recon

KeyboardInterrupt: ignored

In [ ]:
dataset_iter = iter(test_loader)
test_img_0, test_label_0 = next(dataset_iter)

test_img_1, test_label_1 = next(dataset_iter)

In [ ]:
test_img_0 = test_img_0.cuda()
test_img_1 = test_img_1.cuda()
test_label_0 = test_label_0.cuda()
test_label_1 = test_label_1.cuda()

out = model(test_img_0, test_label_0, test_label_1)


In [ ]:
save_image(test_img_0.data, './output/%03d/%04d_img.png' % ( 60, 0))
save_image(out.data, './output/%03d/%04d_recon.png' % ( 60, 0))
save_image(test_img_1.data, './output/%03d/%04d_recon_label.png' % (60, 0))